In [1]:
from allennlp.commands.evaluate import *
from kb.include_all import *
from allennlp.nn import util as nn_util
from allennlp.common.tqdm import Tqdm

import torch
archive_file = "knowbert_wiki_wordnet_model"
cuda_device = 0
# line = "banana\tcolor\tyellow"

/home/scruple/miniconda3/envs/knowbert/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/home/scruple/miniconda3/envs/knowbert/lib/python3.6/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [2]:
sys.modules['torch']

<module 'torch' from '/home/scruple/miniconda3/envs/knowbert/lib/python3.6/site-packages/torch/__init__.py'>

In [3]:
archive = load_archive(archive_file, cuda_device)
config = archive.config
prepare_environment(config)

knowbert_wiki_wordnet_model


In [4]:
config = Params.from_file("/home/scruple/kb/knowbert_wiki_wordnet_model/config.json")

In [5]:
reader_params = config.pop('dataset_reader')
# print(reader_params.as_dict())
if reader_params['type'] == 'multitask_reader':
    reader_params = reader_params['dataset_readers']['language_modeling']
# reader_params['num_workers'] = 0
validation_reader_params = {
    "type": "food",
    "tokenizer_and_candidate_generator": reader_params['base_reader']['tokenizer_and_candidate_generator'].as_dict()
}


In [6]:
dataset_reader = DatasetReader.from_params(Params(validation_reader_params))
vocab = dataset_reader._tokenizer_and_candidate_generator.bert_tokenizer.vocab
token2word = {}
for k, v in vocab.items():
    token2word[v] = k

duplicate_mentions_cnt:  6777
end of p_e_m reading. wall time: 0.7791083494822184  minutes
p_e_m_errors:  0
incompatible_ent_ids:  0


In [7]:
instances = dataset_reader.read("../data/food_compact.tsv")

201it [00:00, 2453.04it/s]


In [8]:
print(instances[69]["tokens"])

TextField of length 8 with text: 
 		[[CLS], constituent, part, of, egg, is, [MASK], [SEP]]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'}


In [9]:
model = archive.model
model.eval()


print("start")
# metrics = evaluate(model, instances, iterator, cuda_device, "")
data_iterator = DataIterator.from_params(Params(
    {"type": "basic", "batch_size": 1}
))
data_iterator.index_with(model.vocab)
iterator = data_iterator(instances,
                            num_epochs=1,
                            shuffle=False)
logger.info("Iterating over dataset")
generator_tqdm = Tqdm.tqdm(iterator, total=data_iterator.get_num_batches(instances))

for instance in generator_tqdm:
    batch = nn_util.move_to_device(instance, cuda_device)
    output_dict = model(**batch)
    pooled_output = output_dict.get("pooled_output")
    contextual_embeddings = output_dict.get("contextual_embeddings")
    prediction_scores, seq_relationship_score = model.pretraining_heads(
    contextual_embeddings, pooled_output
    )
    prediction_scores = prediction_scores.view(-1, prediction_scores.shape[-1])
    print("================")
    # print(prediction_scores)
    print("================")
    idx = torch.argmax(prediction_scores, dim = 1)
    line = ""
    for i in idx.cpu().numpy():
        line += token2word[i] + " "
    print(line)
    

start


  0%|          | 0/201 [00:00<?, ?it/s]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
  0%|          | 1/201 [01:04<3:36:23, 64.92s/it]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp

, label of node ##1 is : ##. . 
the des ##cript ##or of almond is 1 . 
the constituent part of an ##ise is an ##ise , and ##. ##a ##. . 
the constituent part of apple is 1 . 
the constituent part of apple is 1 . 
the constituent part of apple is 1 . 
the constituent part of apple is 1 . 
the des ##cript ##or of apple is 1 . 
the des ##cript ##or of apple is 1 . 
the des ##cript ##or of apple juice is 1 . 


/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
  5%|▌         | 11/201 [01:05<11:16,  3.56s/it]/tmp/pip-req-build-808afw3c/aten/src/ATen/na

the constituent part of art ##ich ##oke is art ##ich ##t language . 
the other part of banana is banana . 
the des ##cript ##or of banana is 1 . 
the constituent part of bay is approximately . 
the constituent part of beer is 1 . 
the constituent part of beer is 1 . 
the des ##cript ##or of beer is 1 . 
##. constituent part of bird ##| ##fo game is p ##. . 
" constituent part of bird ##| ##fo ##wl is af ##. ##. . 
the constituent part of bird ##| ##fo ##wl is 1 . 


...

the constituent part of ta ##co is ta ##co city . 
the constituent part of tea is palm tea . 
the constituent part of toast is called . 
the constituent part of tuna is muscle fish . 
the constituent part of turkey is the ##ı ##k ##ı [UNK] province . 
the constituent part of turkey is k ##. . 
the constituent part of turn ##ip is sweet d ##. . 
the constituent part of walnut is approximately . 


/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
 93%|█████████▎| 187/201 [01:09<00:00, 41.30it/s]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/n

the des ##cript ##or of walnut is 1 . 
the constituent part of it is approximately . 
the constituent part of it is approximately . 
the constituent part of it is approximately . 
the des ##cript ##or of water is water . 
the des ##cript ##or of water is 1 . 
the des ##cript ##or of water is 1 . 
the des ##cript ##or of water is 1 . 
the constituent part of wheat is wheat wheat . 


/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
 98%|█████████▊| 197/201 [01:09<00:00, 40.46it/s]/tmp/pip-req-build-808afw3c/aten/src/ATen/n

the constituent part of white bread is vitamin bread . 
the des ##cript ##or of white chocolate is 1 . 
the constituent part of white sauce is called . 
the constituent part of wine is 1 . 
the constituent part of wine is 1 . 
the constituent part of wine is 1 . 


In [8]:
if reader_params['type'] == 'multitask_reader':
    reader_params = reader_params['dataset_readers']['language_modeling']
# reader_params['num_workers'] = 0
validation_reader_params = {
    "type": "kg_probe",
    "tokenizer_and_candidate_generator": reader_params['base_reader']['tokenizer_and_candidate_generator'].as_dict()
}
dataset_reader = DatasetReader.from_params(Params(validation_reader_params))

vocab = dataset_reader._tokenizer_and_candidate_generator.bert_tokenizer.vocab
token2word = {}
for k, v in vocab.items():
    token2word[v] = k

duplicate_mentions_cnt:  6777
end of p_e_m reading. wall time: 0.793924073378245  minutes
p_e_m_errors:  0
incompatible_ent_ids:  0


/home/scruple/miniconda3/envs/knowbert/lib/python3.6/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [9]:
instances = dataset_reader.read("../data//kg_probe/personCountryOfDeath.text")

392it [00:00, 2029.43it/s]


In [10]:
model = archive.model
model.eval()


print("start")
# metrics = evaluate(model, instances, iterator, cuda_device, "")
data_iterator = DataIterator.from_params(Params(
    {"type": "basic", "batch_size": 1}
))
data_iterator.index_with(model.vocab)
iterator = data_iterator(instances,
                            num_epochs=1,
                            shuffle=False)
logger.info("Iterating over dataset")
generator_tqdm = Tqdm.tqdm(iterator, total=data_iterator.get_num_batches(instances))

for instance in generator_tqdm:
    batch = nn_util.move_to_device(instance, cuda_device)
    output_dict = model(**batch)
    pooled_output = output_dict.get("pooled_output")
    contextual_embeddings = output_dict.get("contextual_embeddings")
    prediction_scores, seq_relationship_score = model.pretraining_heads(
    contextual_embeddings, pooled_output
    )
    prediction_scores = prediction_scores.view(-1, prediction_scores.shape[-1])
    print("================")
    # print(prediction_scores)
    print("================")
    idx = torch.argmax(prediction_scores, dim = 1)
    line = ""
    for i in idx.cpu().numpy():
        line += token2word[i] + " "
    print(line)
    

start


  0%|          | 0/392 [00:00<?, ?it/s]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
  0%|          | 1/392 [01:04<7:02:58, 64.91s/it]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp

the owen mcc ##ar ##ron died in the country of scotland . . 
the tony fall died in the country of mexico . . 
, ge ##ron ##imo pratt died in the country of mexico . . 
the herbert lu ##ms ##den died in the country of england . . 
, lambert ##o v ##. ave ##llan ##a died in the country of spain . . 
the ju ##mbo ts ##uru ##ta died in the country of rwanda . . 
the manuel conde died in the country of spain . . 
the ze ##man no ##a died in the country of egypt . . 
the harry alan towers died in the country of england . . 
the fran ##c fra ##kel ##j died in the country of belgium . . 
, duncan r ##. derry died in the country of ireland . . 


/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning:

the harry va ##il died in the country of england . . 
the sun my ##ung moon died in the country of new korea . . 
, sang lee died in the country of new korea . . 
, william george keith el ##phi ##nst ##one died in the country of england . . 
, abdul ha ##q died in the country of pakistan . . 
the mas ##ah ##iko kim ##ura died in the country of japan . . 
the edo ##gawa ramp ##o died in the country of japan . . 
the fl ##o h ##yman died in the country of germany . . 
the [UNK] died in the country of hungary . . 
, mo ##to ##o kim ##ura died in the country of japan . . 


...

the mor ##de ##chai av ##nie ##l died in the country of israel . . 
the re ##em ri ##yas ##hi died in the country of pakistan . . 
the ali ##yu mud ##a died in the country of turkey . . 
the bar ##uch os ##tro ##vsky died in the country of ukraine . . 
, sheikh ahmed salim sw ##eda ##n died in the country of pakistan . . 
, dharma ##sen ##a path ##ira ##ja died in the country of sri lanka . . 
the lim ye ##w hoc ##k died in the country of new korea . . 
, mohammed bin aw ##ad bin laden died in the country of saudi arabia . . 
, sale ##h abdul aziz al raj ##hi died in the country of saudi arabia . . 
, nad ##ara ##jah ravi ##raj died in the country of sri lanka . . 


/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
 96%|█████████▌| 375/392 [01:13<00:00, 44.92it/s]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/n

, jeanne ho ##ban died in the country of hong kong . . 
the dr ##i family died in the country of tajikistan . . 
the en ##ver pasha died in the country of turkey . . 
the yin shu ##n died in the country of qi . . 
the mar ##on died in the country of france . . 
the han fu ##ju died in the country of korea . . 
the shi ##ge ##ka ##zu shi ##ma ##zaki died in the country of japan . . 
, cicero minor died in the country of italy . . 
the nor ##ia ##ki fu ##ku ##yama died in the country of japan . . 
the anton ##ius ham ##bro ##ek died in the country of holland . . 


/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
 98%|█████████▊| 385/392 [01:13<00:00, 45.29it/s]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/n

the sasha ga ##bor died in the country of croatia . . 
, prince abd ##a died in the country of united arab emirates . . 
, salem sa ##ad died in the country of new united emirates . . 
the jean ##- ##lou ##is bonn ##ard died in the country of paris . . 
the dick ha ##ug ##land died in the country of denmark . . 
the wei chu ##eh died in the country of taiwan . . 
, hum ##bert ro ##que versa ##ce died in the country of paris . . 
the raymond duc died in the country of paris . . 
